# Theni Tamil Dialect Translator — mBART Fine-tune
**Before running:**
1. Runtime → Disconnect and delete runtime
2. Runtime → Change runtime type → T4 GPU → Save
3. Run each cell top to bottom

In [ ]:
# CELL 1 - Install
!pip install -q transformers sentencepiece peft accelerate sacrebleu
print('Done')

In [ ]:
# CELL 2 - Imports
import os, gc, io, torch, shutil, math
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import (
    MBartForConditionalGeneration, MBart50Tokenizer,
    get_cosine_schedule_with_warmup, DataCollatorForSeq2Seq
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from tqdm.notebook import tqdm
import sacrebleu   # pip install sacrebleu

gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    free  = torch.cuda.mem_get_info()[0] / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {total:.1f} GB total | {free:.1f} GB free')


In [ ]:
# CELL 3 - Upload CSV
from google.colab import files
print('Upload your dataset.csv ...')
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df_raw   = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f'Uploaded: {filename}')
print(f'Rows: {len(df_raw)}')
print(f'Columns: {list(df_raw.columns)}')
df_raw.head(3)

In [ ]:
# CELL 4 - Settings
SOURCE_COL = 'normal_tamil'   # change if your CSV column is named differently
TARGET_COL = 'theni_tamil'    # change if your CSV column is named differently

MODEL_NAME = 'facebook/mbart-large-50'
SRC_LANG   = 'ta_IN'
TGT_LANG   = 'ta_IN'

MAX_LENGTH         = 128
BATCH_SIZE         = 8          # physical batch size (fits T4)
GRAD_ACCUM_STEPS   = 4          # effective batch = 8 × 4 = 32
EPOCHS             = 30         # more epochs with early stopping
LR                 = 5e-5       # lower LR → stabler training
LABEL_SMOOTHING    = 0.1        # regularisation
PATIENCE           = 5          # early-stop if val loss doesn't improve
MODEL_DIR          = '/content/best_model'

df = df_raw[[SOURCE_COL, TARGET_COL]].dropna()
df[SOURCE_COL] = df[SOURCE_COL].str.strip()
df[TARGET_COL] = df[TARGET_COL].str.strip()
df = df[(df[SOURCE_COL].str.len() > 0) & (df[TARGET_COL].str.len() > 0)]

# ── Data augmentation: duplicate rare / short pairs ─────────────────────────
df_aug = df.copy()
# Duplicate pairs where source has fewer than 5 words (harder short sentences)
short_mask = df[SOURCE_COL].str.split().str.len() < 5
df_aug = pd.concat([df, df[short_mask]], ignore_index=True)
print(f'Original pairs: {len(df)} | After augmentation: {len(df_aug)}')

src_train, src_val, tgt_train, tgt_val = train_test_split(
    df_aug[SOURCE_COL].tolist(), df_aug[TARGET_COL].tolist(),
    test_size=0.1, random_state=42
)
print(f'Train: {len(src_train)} | Val: {len(src_val)}')


In [ ]:
# CELL 5 - Load mBART + LoRA
gc.collect()
torch.cuda.empty_cache()

print('Loading tokenizer...')
tokenizer = MBart50Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG

print('Loading base model in fp16...')
base_model = MBartForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16
)
base_model.config.use_cache = False

print('Applying LoRA...')
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,                          # ↑ from 16 → 32 (more capacity)
    lora_alpha=64,                 # keep alpha = 2×r
    lora_dropout=0.05,             # small dropout
    # ── target BOTH encoder AND decoder attention + FFN gates ────────────
    target_modules=[
        'q_proj', 'k_proj', 'v_proj', 'out_proj',   # attention
        'fc1', 'fc2'                                  # feed-forward
    ],
    bias='none',
)
model = get_peft_model(base_model, lora_config)

# Cast LoRA params to float32 for stable gradients with fp16 base
for name, param in model.named_parameters():
    if param.requires_grad:
        param.data = param.data.float()

model.to(device)

total     = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total params    : {total/1e6:.1f}M')
print(f'Trainable params: {trainable/1e6:.2f}M ({100*trainable/total:.2f}%)')
free = torch.cuda.mem_get_info()[0] / 1e9
print(f'Free VRAM       : {free:.1f} GB')


In [ ]:
# CELL 6 - Dataset  (dynamic padding via DataCollatorForSeq2Seq)
class TheniDataset(Dataset):
    def __init__(self, sources, targets, tokenizer, max_length=128):
        self.sources   = sources
        self.targets   = targets
        self.tokenizer = tokenizer
        self.max_len   = max_length

    def __len__(self):
        return len(self.sources)

    def __getitem__(self, idx):
        # Encode source
        self.tokenizer.src_lang = SRC_LANG
        model_inputs = self.tokenizer(
            str(self.sources[idx]),
            max_length=self.max_len,
            truncation=True,
        )
        # Encode target: MBart50Tokenizer uses tgt_lang to set the BOS token
        self.tokenizer.src_lang = TGT_LANG
        labels = self.tokenizer(
            str(self.targets[idx]),
            max_length=self.max_len,
            truncation=True,
        )
        self.tokenizer.src_lang = SRC_LANG  # restore
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

# Dynamic padding collator — pads each batch to its longest sequence
collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,   # -100 is ignored in loss
    pad_to_multiple_of=8,      # good for tensor-core alignment
)

train_dataset = TheniDataset(src_train, tgt_train, tokenizer, MAX_LENGTH)
val_dataset   = TheniDataset(src_val,   tgt_val,   tokenizer, MAX_LENGTH)
train_loader  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                           collate_fn=collator, num_workers=2, pin_memory=True)
val_loader    = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False,
                           collate_fn=collator, num_workers=2, pin_memory=True)
print(f'Steps/epoch: {len(train_loader)}  |  Eff. batch size: {BATCH_SIZE * GRAD_ACCUM_STEPS}')


In [ ]:
# CELL 7 - Train  (label smoothing + cosine LR + gradient accumulation + early stopping + BLEU)
import torch.nn.functional as F

def smooth_loss(logits, labels, eps=LABEL_SMOOTHING):
    """Cross-entropy with label smoothing; ignores -100 positions."""
    vocab_size = logits.size(-1)
    # flat
    logits_flat = logits.view(-1, vocab_size)
    labels_flat = labels.view(-1)

    mask = labels_flat != -100
    logits_flat = logits_flat[mask]
    labels_flat = labels_flat[mask]

    log_probs = F.log_softmax(logits_flat, dim=-1)
    nll = F.nll_loss(log_probs, labels_flat, reduction='mean')
    smooth = -log_probs.mean()
    return (1 - eps) * nll + eps * smooth

total_steps  = len(train_loader) // GRAD_ACCUM_STEPS * EPOCHS
warmup_steps = total_steps // 10   # 10 % warm-up

optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=LR, weight_decay=0.01, betas=(0.9, 0.98)
)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

os.makedirs(MODEL_DIR, exist_ok=True)
best_val_loss   = float('inf')
patience_count  = 0
history         = []

print(f'Training {EPOCHS} epochs | {total_steps} total opt-steps | '
      f'{warmup_steps} warmup steps')

for epoch in range(1, EPOCHS + 1):
    # ── Train ───────────────────────────────────────────────────────────────
    model.train()
    total_train = 0
    optimizer.zero_grad()

    for step, batch in enumerate(tqdm(train_loader, desc=f'Epoch {epoch}/{EPOCHS} [Train]')):
        input_ids      = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels         = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        # Use label-smoothed loss instead of model's raw CE
        loss = smooth_loss(outputs.logits, labels) / GRAD_ACCUM_STEPS
        total_train += loss.item() * GRAD_ACCUM_STEPS
        loss.backward()

        if (step + 1) % GRAD_ACCUM_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    # ── Validate  (loss + BLEU/chrF) ────────────────────────────────────────
    model.eval()
    total_val    = 0
    hyps, refs   = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f'Epoch {epoch}/{EPOCHS} [Val]'):
            input_ids      = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels         = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            total_val += smooth_loss(outputs.logits, labels).item()

            # ── Generate for BLEU (every epoch) ─────────────────────────────
            forced_bos = tokenizer.lang_code_to_id[TGT_LANG]
            gen_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                forced_bos_token_id=forced_bos,
                num_beams=4,
                max_length=MAX_LENGTH,
                early_stopping=True
            )
            decoded_hyp = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
            # Recover reference strings (replace -100 back to pad_id for decoding)
            ref_ids = labels.clone()
            ref_ids[ref_ids == -100] = tokenizer.pad_token_id
            decoded_ref = tokenizer.batch_decode(ref_ids, skip_special_tokens=True)

            hyps.extend(decoded_hyp)
            refs.extend(decoded_ref)

            del input_ids, attention_mask, labels, outputs, gen_ids
            torch.cuda.empty_cache()

    avg_train = total_train / len(train_loader)
    avg_val   = total_val   / len(val_loader)

    bleu_score = sacrebleu.corpus_bleu(hyps, [refs]).score
    chrf_score = sacrebleu.corpus_chrf(hyps, [refs]).score

    history.append({'epoch': epoch, 'train': avg_train, 'val': avg_val,
                    'bleu': bleu_score, 'chrf': chrf_score})

    print(f'Epoch {epoch:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f} '
          f'| BLEU: {bleu_score:.2f} | chrF: {chrf_score:.2f}')

    if avg_val < best_val_loss:
        best_val_loss  = avg_val
        patience_count = 0
        model.save_pretrained(MODEL_DIR)
        tokenizer.save_pretrained(MODEL_DIR)
        print(f'  ✓ Saved best model (val={best_val_loss:.4f}, BLEU={bleu_score:.2f})')
    else:
        patience_count += 1
        print(f'  No improvement ({patience_count}/{PATIENCE})')
        if patience_count >= PATIENCE:
            print('Early stopping triggered.')
            break

print('\nTraining complete!')
print(f'Best val loss: {best_val_loss:.4f}')

# ── Print training curve ─────────────────────────────────────────────────────
print('\nEpoch | Train  | Val    | BLEU  | chrF')
print('-'*45)
for h in history:
    print(f"{h['epoch']:5d} | {h['train']:.4f} | {h['val']:.4f} | {h['bleu']:5.2f} | {h['chrf']:.2f}")


In [ ]:
# CELL 8 - Download model
from google.colab import files
shutil.make_archive('/content/theni_mbart_model', 'zip', MODEL_DIR)
files.download('/content/theni_mbart_model.zip')
print('Downloaded!')

In [ ]:
# CELL 9 - Load model for translation
gc.collect()
torch.cuda.empty_cache()

# MBart50Tokenizer must be used (not AutoTokenizer) to preserve lang_code_to_id
tokenizer_t = MBart50Tokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
tokenizer_t.src_lang = SRC_LANG
tokenizer_t.tgt_lang = TGT_LANG

base_t  = MBartForConditionalGeneration.from_pretrained(MODEL_NAME, torch_dtype=torch.float16)
model_t = PeftModel.from_pretrained(base_t, MODEL_DIR, local_files_only=True)
model_t = model_t.merge_and_unload()
model_t.to(device)
model_t.eval()
print('Model ready!')

def translate(text):
    tokenizer_t.src_lang = SRC_LANG
    inputs = tokenizer_t(
        text, return_tensors='pt',
        max_length=MAX_LENGTH, truncation=True
    ).to(device)
    # For mBART, forced_bos_token_id must be the target language token id
    forced_bos_token_id = tokenizer_t.lang_code_to_id[TGT_LANG]
    with torch.no_grad():
        out = model_t.generate(
            **inputs,
            forced_bos_token_id=forced_bos_token_id,
            num_beams=5,
            max_length=MAX_LENGTH,
            early_stopping=True
        )
    return tokenizer_t.decode(out[0], skip_special_tokens=True)

tests = [
    'நீங்கள் எப்படி இருக்கிறீர்கள்',
    'இன்று மழை பெய்கிறது',
    'நான் சாப்பிட போகிறேன்',
    'அவர் வீட்டிற்கு வருகிறார்'
]
print('\n--- Test Results ---')
for t in tests:
    print(f'Normal Tamil : {t}')
    print(f'Theni Slang  : {translate(t)}')
    print('-' * 50)

In [ ]:
# CELL 10 - Interactive translation
print('Type any Normal Tamil sentence. Type quit to stop.\n')
while True:
    text = input('Normal Tamil : ').strip()
    if text.lower() == 'quit':
        break
    if text:
        print(f'Theni Slang  : {translate(text)}\n')